In [1]:
import pandas as pd
from IPython.display import Image, HTML
from datetime import datetime

In [2]:
# convert link to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [3]:
# Grab raw data to convert to html pages.
dfs = pd.read_csv("inputs/dfs.csv")
fd = pd.read_csv("inputs/fanduel.csv")
dk = pd.read_csv("inputs/draftkings.csv")
st = pd.read_csv("inputs/statline.csv")
dfs = dfs.drop("Unnamed: 0", axis=1)
fd = fd.drop("Unnamed: 0", axis=1)
dk = dk.drop("Unnamed: 0", axis=1)
st = st.drop("Unnamed: 0", axis=1)

In [4]:
# Grab timestamp.
ts = datetime.now()
ts = ts.strftime("%b %d, %Y %I:%M %p")
ts = "Last Updated: " + ts

In [5]:
# Highlights given player row to indicate hot/cold status for 19 columns.
def highlight_dfs(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*19
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*19
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*19
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*19
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*19
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*19
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*19
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*19
    else: return ['background-color: white']*19

In [6]:
# Highlights given player row to indicate hot/cold status for 14 columns.
def highlight_fd(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*14
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*14
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*14
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*14
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*14
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*14
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*14
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*14
    else: return ['background-color: white']*14

In [7]:
# Adds fire or ice emoji for respective hot/cold status.
def emojize(df):
    for i in range(len(df)):
        if df['Hot'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n🔥')
        if df['Hot'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Hot'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Cold'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n❄️')
        if df['Cold'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
        if df['Cold'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
    return df

In [8]:
# Error Handling

# Players that don't have a BBall Ref picture, as of 8/7/20.
no_pic_list = ["Paul Watson", "Donta Hall"]

# Player names that don't match to our in-house data.
special_names =       {"Luka Dončić": "Luka Doncic", 
                       "Luka Šamanić": "Luka Samanic", 
                       "Kristaps Porziņģis": "Kristaps Porzingis", 
                       "Nikola Vučević": "Nikola Vucevic",
                       "Jonas Valančiūnas": "Jonas Valanciunas",
                       "Bogdan Bogdanović": "Bogdan Bogdanovic",
                       "Dario Šarić": "Dario Saric",
                       "Timothé Luwawu-Cabarrot": "Timothe Luwawu-Cabarrot",
                       "Džanan Musa": "Dzanan Musa",
                        "Dāvis Bertāns": "Davis Bertans",
                        "Boban Marjanović": "Boban Marjanovic",
                        "Ersan İlyasova": "Ersan Ilyasova",
                        "Anžejs Pasečņiks": "Anzejs Pasecniks",
                       "Bojan Bogdanović": "Bojan Bogdanovic",
                        "Nicolò Melli": "Nicolo Melli",
                        "Nikola Jokić": "Nikola Jokic",
                        "Jusuf Nurkić": "Jusuf Nurkic",
                        "Goran Dragić": "Goran Dragic",
                        "Dennis Schröder" :"Dennis Schroder",
                       "Gary Payton": "Gary Payton II",
                       "Mohamed Bamba": "Mo Bamba",
                       "Wesley Iwundu": "Wes Iwundu",
                        "J.J. Redick": "JJ Redick",
                        "B.J. Johnson": "BJ Johnson"}
# Invert dictionary for use in data grabbing.
inv_names = {v: k for k, v in special_names.items()}

inv_names

{'Luka Doncic': 'Luka Dončić',
 'Luka Samanic': 'Luka Šamanić',
 'Kristaps Porzingis': 'Kristaps Porziņģis',
 'Nikola Vucevic': 'Nikola Vučević',
 'Jonas Valanciunas': 'Jonas Valančiūnas',
 'Bogdan Bogdanovic': 'Bogdan Bogdanović',
 'Dario Saric': 'Dario Šarić',
 'Timothe Luwawu-Cabarrot': 'Timothé Luwawu-Cabarrot',
 'Dzanan Musa': 'Džanan Musa',
 'Davis Bertans': 'Dāvis Bertāns',
 'Boban Marjanovic': 'Boban Marjanović',
 'Ersan Ilyasova': 'Ersan İlyasova',
 'Anzejs Pasecniks': 'Anžejs Pasečņiks',
 'Bojan Bogdanovic': 'Bojan Bogdanović',
 'Nicolo Melli': 'Nicolò Melli',
 'Nikola Jokic': 'Nikola Jokić',
 'Jusuf Nurkic': 'Jusuf Nurkić',
 'Goran Dragic': 'Goran Dragić',
 'Dennis Schroder': 'Dennis Schröder',
 'Gary Payton II': 'Gary Payton',
 'Mo Bamba': 'Mohamed Bamba',
 'Wes Iwundu': 'Wesley Iwundu',
 'JJ Redick': 'J.J. Redick',
 'BJ Johnson': 'B.J. Johnson'}

In [9]:
# Creates an HTML page based on input DataFrame and respective coloring.
'''
Args: 
    df: input DataFrame
    out: name of output HTML file (refer to main pages for correct naming)
    coloring: hot/cold highlights for HTML page
'''
def gen_page(df, out, coloring):
    # Read games file to grab slug code, for BBall Ref data.
    games = pd.read_csv('games.csv').loc[:,['name','slug']]
    trans_games = games.set_index('name').T
    img_dict = trans_games.to_dict('list')
    
    # Update data in Name and Picture column to display link to BBall Ref page and profile picture, respectively.
    images = []
    names = []
    for i in df["Name"]:
        display_name = i
        if i in inv_names:
            display_name = inv_names[i]
        code = "https://raw.githubusercontent.com/BerkeleySportsAnalytics/sagb-site/master/images/profile_picture.jpg"
        # Catch errors when player is not tracked in games file.
        if img_dict.get(display_name) != None:
            if display_name not in no_pic_list:
                # BBall Ref image hyperlink.
                code = "https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/" + img_dict.get(display_name)[0] + ".jpg"
            # BBall Ref player profile hyperlink and embed code.
            display_name = "<a href=\"https://www.basketball-reference.com/players/" + \
                img_dict.get(display_name)[0][0] + "/" + img_dict.get(display_name)[0] + ".html\" target=\"_blank\">" + display_name + "</a>"
        images.append(code)
        names.append(display_name)
    df["Picture"] = images
    df["Name"] = names
    
    # Change the picture column to the second column in order.
    cols = df.columns.tolist()
    cols = [cols[0]] + cols[-1:] + cols[1:-1]
    df = df[cols]
    pd.set_option('display.max_colwidth', -1)
    df.index +=1
    
    # Header of output HTML file.
    pre = (
        "<head>\n<script src=\"sorttable.js\"></script>\n"
        "<link rel=\"stylesheet\" href=\"df_styles.css\">\n</head>\n"
    )
    pre = pre + ts + "\n"
    pre += (
        "\n<body>\n<div id=\"filter\">\n<input type=\"text\" id=\"name\" onkeyup=\"nameSearch()\" placeholder=\"Search for name...\">\n"
        "<input type=\"text\" id=\"team\" onkeyup=\"teamSearch()\" placeholder=\"Search for team...\">\n"
        "<label for=\"pos\">Search by position:</label>\n"
        "<select id=\"pos\" onchange=\"posSearch()\" class='form-control'>\n"
        "<option></option>\n"
        "<option>PG</option>\n"
        "<option>SG</option>\n"
        "<option>SF</option>\n"
        "<option>PF</option>\n"
        "<option>C</option>\n"
        "</select>\n"
        "</div>\n\n"
    )
    
    # Footer of output HTML file.
    post = (
        "\n"
        "\n<script src=\"main.js\"></script>"
        "\n</body>"
        )
    
    # Render dataframe as HTML.
    if coloring is not None:
        images = ['<img src="' + image + '" width="60">' for image in images]
        df["Picture"] = images
        df = emojize(df)
        body = df.style.apply(coloring, axis=1).hide_columns(['Hot', 'Cold']).render()
    else:
        html = HTML(df.to_html(escape=False ,formatters=dict(Picture=path_to_image_html)))
        body = html.data

    # Generate HTML and add configuration for output tables, so JavaScript functions can run properly.
    body = body.replace("<table id=", "<table ignore=", 1)
    body = body.replace("<table ", "<table id=\"tbl\" class=\"sortable\" ", 1)
    a = pre + body + post
    # write html to file
    with open(out, 'w', encoding="utf-8") as f:
        f.write(a)
    return df

In [10]:
# Create the page for given output.
gen_page(dfs, 'dfs.html', highlight_dfs)
gen_page(fd, 'fd.html', highlight_fd)
gen_page(dk, 'dk.html', highlight_fd)
gen_page(st, 'st.html', None)

<ipython-input-9-1d3efe937277>:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  img_dict = trans_games.to_dict('list')
<ipython-input-9-1d3efe937277>:39: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
<ipython-input-7-4925ea65e2bc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if df['Cold'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n❄️')
<ipython-input-7-4925ea65e2bc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Name,Picture,Position,Game,Team,Minutes,2PT FG,3PT FG,FTM,Rebounds,Assists,Blocks,Steals,Turnovers,Opponent Defensive Rank vs Position,Injury Indicator,Injury Details
1,"<a href=""https://www.basketball-reference.com/players/a/antetgi01.html"" target=""_blank"">Giannis Antetokounmpo</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/antetgi01.jpg,SF,MIL@ORL,MIL,30.22,10.31,0.87,6.52,12.61,5.07,1.34,1.88,4.14,15.0,,
2,"<a href=""https://www.basketball-reference.com/players/h/hardeja01.html"" target=""_blank"">James Harden</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/hardeja01.jpg,SG,HOU@OKC,HOU,36.07,8.19,3.92,10.83,6.40,8.61,0.70,2.05,5.05,11.0,,
3,"<a href=""https://www.basketball-reference.com/players/d/davisan02.html"" target=""_blank"">Anthony Davis</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/davisan02.jpg,PF,LAL@POR,LAL,42.52,10.17,0.22,4.70,6.58,3.27,1.61,1.90,1.59,29.0,,
4,"<a href=""https://www.basketball-reference.com/players/j/jamesle01.html"" target=""_blank"">LeBron James</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/jamesle01.jpg,SF,LAL@POR,LAL,42.29,7.67,3.82,6.15,9.06,10.49,0.83,2.21,4.19,13.0,,
5,"<a href=""https://www.basketball-reference.com/players/l/lillada01.html"" target=""_blank"">Damian Lillard</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/lillada01.jpg,PG,LAL@POR,POR,38.96,3.81,4.33,5.14,4.71,6.87,0.00,1.29,3.91,11.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,"<a href=""https://www.basketball-reference.com/players/s/simonan01.html"" target=""_blank"">Anfernee Simons</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/simonan01.jpg,PG,LAL@POR,POR,17.87,0.87,0.48,0.42,1.57,0.29,0.18,0.38,0.79,10.0,,
129,"<a href=""https://www.basketball-reference.com/players/b/bowenbr02.html"" target=""_blank"">Brian Bowen</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/bowenbr02.jpg,SG,IND@MIA,IND,20.72,0.78,0.00,0.36,2.24,0.05,0.91,0.00,0.00,19.0,,
130,"<a href=""https://www.basketball-reference.com/players/j/johnsal02.html"" target=""_blank"">Alize Johnson</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/johnsal02.jpg,PF,IND@MIA,IND,18.10,2.00,0.00,0.83,1.71,0.29,0.94,0.04,0.00,6.0,,
131,"<a href=""https://www.basketball-reference.com/players/f/frazime01.html"" target=""_blank"">Melvin Frazier</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/frazime01.jpg,SF,MIL@ORL,ORL,17.61,0.61,0.00,0.70,1.52,0.00,0.77,0.14,0.00,8.0,GTD,Back
